In [2]:
import cv2
import os
import torch
from transformers import DetrFeatureExtractor, DetrForObjectDetection
from tqdm import tqdm
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device used here is: {device}")

c:\Users\imad-\Desktop\Sat det\code\sat\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device used here is: cuda


In [2]:
import torch

print("CUDA disponible :", torch.cuda.is_available())
print("Nom du GPU :", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Aucun")


CUDA disponible : True
Nom du GPU : NVIDIA GeForce MX130


In [ ]:
# Chemins
image_dir = "output_tiles"
images = [f for f in os.listdir(image_dir) if f.endswith((".jpg", ".tif"))]
label_dir = "output_tiles/output_label_DETR"
os.makedirs(label_dir, exist_ok=True)

In [4]:
# Charger le modèle DETR
feature_extractor = DetrFeatureExtractor.from_pretrained('facebook/detr-resnet-50')
model = DetrForObjectDetection.from_pretrained('facebook/detr-resnet-50')

c:\Users\imad-\Desktop\Sat det\code\sat\lib\site-packages\transformers\models\detr\feature_extraction_detr.py:38: FutureWarning: The class DetrFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use DetrImageProcessor instead.
  warnings.warn(
Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initiali

In [9]:
# Liste des classes COCO
coco_classes = [
    'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
    'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
    'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
    'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
    'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
    'hair drier', 'toothbrush'
]

In [5]:
def detect_objects(image):
    # Prétraitement de l'image
    inputs = feature_extractor(images=image, return_tensors="pt")
    # Inférence
    outputs = model(**inputs)
    # Post-traitement des résultats
    results = feature_extractor.post_process_object_detection(outputs, threshold=0.5, target_sizes=[image.shape[:2]])
    return results[0]

In [6]:
def generate_yolo_annotation(image, detections, classes, output_dir):
    image_height, image_width = image.shape[:2]
    for detection in detections:
        class_id = classes.index(detection['label'])
        x_center = detection['box']['center_x'] / image_width
        y_center = detection['box']['center_y'] / image_height
        width = detection['box']['width'] / image_width
        height = detection['box']['height'] / image_height
        annotation = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n"
        with open(os.path.join(output_dir, f"image_{i}.txt"), 'a') as f:
            f.write(annotation)


In [7]:
# Détecter les objets dans chaque image et générer les annotations
for img_file in tqdm(images):
    img_path = os.path.join(image_dir, img_file)
    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    # Inférence
    results = detect_objects(img)

    # Fichier d’annotation à écrire
    label_path = os.path.join(label_dir, img_file.replace(".jpg", ".txt").replace(".tif", ".txt"))

    with open(label_path, "w") as f:
        for i, detection in enumerate(results['boxes']):
            # Récupérer les coordonnées de la boîte englobante
            x1, y1, x2, y2 = detection.tolist()
            # Récupérer l'indice de la classe
            class_id = int(results['labels'][i])
            # Récupérer la probabilité de la classe (optionnel)
            score = results['scores'][i]

            # Calcul des valeurs normalisées YOLOv8
            x_center = ((x1 + x2) / 2) / w
            y_center = ((y1 + y2) / 2) / h
            box_w = (x2 - x1) / w
            box_h = (y2 - y1) / h

            # Ligne formatée YOLO
            line = f"{class_id} {x_center:.6f} {y_center:.6f} {box_w:.6f} {box_h:.6f}"
            f.write(line + "\n")

  0%|          | 0/1783 [00:00<?, ?it/s]

100%|██████████| 1783/1783 [56:46<00:00,  1.91s/it]
